![](blackjack.jpg)

# Reinforcement Learning: Blackjack and FrozenLake

In this lab, we are going to implement and study some simple tabular model-free RL methods. In particular, we will work with BlackJack and Frozenlake.

This lab consists of two parts, which are similar in nature. In part one, we will try to develop a learning algorithm to learn a good policy for black jack. And part in two, we will extend this to using SARSA and Q-Learning for Frozen Lake. The difference is that Frozen Lake has much longer episodes, so that it makes sense to learn during the episodes, instead of only after the episode finishes. Note that part two of this lab is in a separate notebook.

# Intro to Reinforcement Learning

The field of Reinforcement Learning (RL) is full of various terms and algorithms that all try to do roughly the same thing. That is, to learn an optimal policy in a state-action setting. This can sometimes, be a little confusing, so let us here first with orienting ourselves. We often first distinguish between model-based and model-free RL, where the model in this case is a model of the state transitions. In some problems, we assume that we know the underlying Markov Decision Process (MDP) and its state transitions and rewards. In such cases, we can use value iteration or policy iteration to theoretically calculate the optimal policy.

In this lab, even though we theoretically know the underlying model, we will assume that it is unknown to us, and that we will have to learn the underlying policy from experience. In model-free problems, it is often easier to work with state-action values, normally called `Q`, instead of the state value function, often denoted by `V`.

There is also a question whether to update the policy during an episode, or wait until the episode has finished. We will use completed episodes in the BlackJack example, and implement SARSA and Q-Learning for the FrozenLake problem.

## Part 1: Black Jack

### Rules of the Game
The object of the popular casino card game of blackjack is to obtain cards the sum of whose numerical values is as great as possible without exceeding 21. All face cards count as 10, and an ace can count either as 1 or as 11. We consider the version in which each player competes independently against the dealer. The game begins with two cards dealt to both dealer and player. One of the dealer’s cards is face up and the other is face down. If the player has 21 immediately (an ace and a 10-card), it is called a natural. They then immediately wins. If the player does not have a natural, then they can request additional cards, one by one (hits), until they either stops (sticks) or exceeds 21 (goes bust). If the player goes bust, they lose; if they stick, then it becomes the dealer’s turn. The dealer hits or sticks according to a fixed strategy without choice: they stick on any sum of 17 or greater, and hits otherwise. If the dealer goes bust, then the player wins; otherwise, the outcome—win, lose, or draw—is determined by whose final sum is closer to 21, with the dealer winning draws.

### Blackjack in Reinforcement Learning
Playing blackjack is naturally formulated as an episodic finite MDP. Each game of blackjack is an episode. Rewards of +1, -1, and 0 are given for winning, losing, and drawing, respectively. All rewards within a game are zero, and we do not discount ( gamma = 1); therefore these terminal rewards are also the returns. The player’s actions are to hit or to stick. The states depend on the player’s cards and the dealer’s showing card. We assume that cards are dealt from an infinite deck (i.e., with replacement) so that there is no advantage to keeping track of the cards already dealt. As aces can be counted as either 1 or 11, we also need to keep track of the players has a usable ace. Usable in this case means that it is currently counted as 11. If the player would go bust if we count it as 11, we instead count is one and say that they no longer has a usable ace.

Hence, we have the following state space: The player's current score (12–21), the dealer’s one showing card (ace–10), and whether or not the player holds a usable ace. This makes for a total of 200 states.

The action space, on the other hand is only stick or hit, or 0 or 1.

# Getting acquainted with the environment

It is common practice in Reinforcement Learning to define your problem as a 'gym' to keep a consistent format. A gym is a class with a few common elements:
- An action space, listing the available actions for the **user**
- A state space, listing the format of the problem states
- A `reset()` function that restarts and initiates an episode
- A `step()` function that takes an action argument and returns the reward and the new state after the action has been performed


Gymnasium comes with a set of predefined environments ([https://gymnasium.farama.org/index.html](https://gymnasium.farama.org/index.html)), but you can also create your own.

In this case, we have our own gym which is defined in [blackjack.py](blackjack.py).

Before we start with the reinforcement learning, let us quickly get acquainted with the blackjack environment.

In [1]:
# import some basic packages
import plotting
import numpy as np
import numpy.typing as npt
from blackjack import BlackjackEnv
import matplotlib
import time
from collections import defaultdict
matplotlib.style.use('ggplot')

In [2]:
# create the env
env = BlackjackEnv()

# Playing around with the Env

Here we provide some code defines a simple strategy and tests it against the environment. Before starting the lab make certain you understand how the environment works, and how the observation and actions spaces look like.

In [ ]:
def print_observation(obs: tuple[int, int, int]):
    """Prints an observation in a prettier format."""
    score, dealer_score, usable_ace = obs
    print(
        "Player Score: {} (Usable Ace: {}), Dealer Score: {}".format(
          score, usable_ace, dealer_score
        )
    )

def strategy(obs: tuple[int, int, int]) -> int: # return 0 or 1
    """A strategy takes an observation and returns an action"""
    score, dealer_score, usable_ace = observation

    # ➡️ TODO : implement some simple one-line strategy for whether to hit or not ⬅️
    return ...

# Lastly we put your strategy to the test
reward_sum = 0
n_iter = 100
for i_episode in range(n_iter): # loop over the episodes
    observation = env.reset()
    done = False
    reward = 0
    while not done:
        print_observation(observation)
        action = strategy(observation)
        print("Taking action: {}".format( ["Stick", "Hit"][action]))
        observation, reward, done, _ = env.step(action)
    print_observation(observation)
    print("Game end. Reward: {}\n".format(float(reward)))
    reward_sum += float(reward)

print("Your strategy had an average reward of:", reward_sum / n_iter)

# Implement a first Q-Learning algorithm

We will use tabular Q-Learning for this problem, meaning that define a value for each state-action pair $q(s,a)$.

Q will be stored in the following format:
```
Q: dict[tuple[int, int, int], npt.NDArray] = defaultdict(lambda: np.zeros(2)),
```

i.e., the keys of this dict are the state tuples, and the values of this dict are np arrays with two values, one for each action.

## Create the Epsilon Greedy Policy

First we implement the greedy policy. A policy in this is just a function that takes the state as input and returns the probability to select the different actions. For example, a policy that always stays would return `np.array([1, 0])`.

Here, we generate a policy factory, i.e., a function that generates policy functions given the current Q-values and an epsilon.

In [ ]:
def epsilon_greedy_policy(
        state: tuple[int, int, int],
        Q: dict[tuple[int, int, int], npt.NDArray],
        epsilon: float,
        nA: int,
) -> npt.NDArray:
    """
    Implements an epsilon-greedy policy based on a given Q-function and epsilon.

    Args:
        state (tuple[int, int, int]): The current state of the environment.
        Q (dict[tuple[int, int, int], npt.NDArray]): A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA
        epsilon (float): The probability to select a random action . float between 0 and 1.
        nA (int): Number of actions in the environment.

    Returns:
        npt.NDArray: An array of length 2 with the probabilities to select the different actions.
    """

    return ...

# The training loop

Next, we train our agent using Q-learning. Here you need to implement the full training loop - iteratively generating new episodes and using them to learn your policy function Q. Note that we want $\varepsilon_k \rightarrow 0$ as $k\rightarrow \infty$.

In this part, we do not need to do any TD learning - the episodes are short and never loop back to previous states so we can just update the Q function as the episode is completed.

In [3]:
def mc_control_epsilon_greedy(
        env: BlackjackEnv,
        num_episodes: int,
        epsilon: float = 0.1,
):
    """
    Monte Carlo Control using Epsilon-Greedy policies.

    Args:
        env (BlackjackEnv): OpenAI gym environment.
        num_episodes (int): Number of episodes to sample.
        epsilon (float): Chance to sample a random action. Float between 0 and 1.

    Returns:
        Q (dict[tuple[int, int, int], npt.NDArray]): a dictionary with the Q values.
        policy (Callable): a function that takes an observation as an argument and returns
        action probabilities.
    """

    # To estimate the Q values, we keep track of the number of times we have visited
    # each state-action-pair, as well as the cumulative reward. We could alternatively,
    # store every individual event, but that is much less memory-efficient.

    # We leave it up to you define the format of the keys for those.
    rewards_sum = defaultdict(float)
    rewards_count = defaultdict(float)

    # Initialize the state-action values
    initial_value = 0
    Q: dict[tuple[int, int, int], npt.NDArray] = defaultdict(lambda: np.ones(env.nA) * initial_value)

    # ➡️ TODO : use the epsilon_greedy_policy_factory defined above and create a training loop
    #    TODO :     that generates new data and gradually updates the Q-function. ⬅️

    return Q, policy

__Questions__ (you might need to run some test through the test code below):
- What is the impact of changing `initial_value`? Why would we want to set it higher/lower?
- How fast should we let $\varepsilon$ decay?
- How many iterations do we need for this to converge?


Let's take it for a spin and see if it works!

In [ ]:
start = time.process_time()

Q, policy = mc_control_epsilon_greedy(env, num_episodes=5000, epsilon=0.1)

end = time.process_time()
print("\n", "your code ran in", end - start,"s")

# Run, test and plot

Finally, we run our training and plot the state reward surface. We recommend playing around with num_episodes, epsilon, initial_value as well as the epsilon decay strategy.

In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, actions in Q.items():
    action_value = np.max(actions)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")

__Questions__:
- Once you have a well working policy, what is the expected reward (before you are dealt your hand) of playing a game of Blackjack? Use your policy to get an estimate.
- Often, the plot for "usable Ace" look more jagged than the one without - why is that?


Well done! On to part 2!